## Module 3 lab - KMeans Clustering

We have discussed clustering in Statmath course. You can [revisit the course here](https://jupyterhub.dsa.missouri.edu/user/skaf48/tree/DSA-8610/modules/module7). As an unsupervised data analysis technique, It organises data objects by proximity based on its variables and helps to create natural groupings for a set of data objects. By grouping data one can understand how each data point relates to each other and discover groups of similar ones. 

When the groups are formed, centroids can be defined for them, the ideal data object that minimises the sum of the distances to each of the data points in a cluster. By analysing these centroids variables we will be able to define each cluster in terms of its characteristics.

In this notebook, the dataset is related to deaths from Infectious Tuberculosis disease in each country from 1990 to 2007. The data is available on [gapminder website](http://www.gapminder.org/data/) which is a comprehensive resource for data regarding different countries and territories indicators. We will work with data where each sample is a country and each variable is a year. 

Kmeans clustering divides set of data objects into k clusters, assigning each observation to a cluster so as to minimize the distance of that observation (in n-dimensional space) to the cluster’s mean; the means are then recomputed. This operation is run iteratively until the clusters converge, for a maximum of number of iterations chosen. Given a target number, k, of clusters to find, it will locate the centers of each of those k clusters and the boundaries between them. It does this using the following algorithm:

1. Start with a randomly selected set of  k  centroids (the supposed centers of the  k  clusters)
2. Determine which observation is in which cluster, based on which centroid it is closest to (using the squared Euclidean distance:  ∑pj=1(xij−xi′j)2  where  p  is the number of dimensions)
3. Re-calculate the centroids of each cluster by minimizing the squared Euclidean distance to each observation in the cluster
4. Repeat 2. and 3. until the members of the clusters (and hence the positions of the centroids) no longer change.

In [ ]:
import pandas as pd

tb_deaths_per_100k = pd.read_csv("../../../datasets/TB/tb_deaths_per_100k.csv",index_col = 0, thousands  = ',')
tb_deaths_per_100k.index.names = ['country']
tb_deaths_per_100k.columns.names = ['year']
tb_deaths_per_100k

An extra row of NANs' is introduced. Get rid of that row using pandas indexing as shown below. Also get rid of the extra column "Unnames: 19" that is introduced at the end. 

In [ ]:
# Exclude last row of data
tb_deaths_per_100k = tb_deaths_per_100k[:-1]

# Exclude last column with NANs'
del tb_deaths_per_100k["Unnamed: 19"]

In [ ]:
tb_deaths_per_100k

Kmeans takes set of data points and clusters them in specified number of groups. The data here has 18 variables. Use PCA to reduce the dimensions of the data. Python's sklearn machine learning library comes with a PCA implementation. 

For very large dimensional data, normal PCA implementation may not work. We should consider apache Spark's dimensionality reduction features that is explored in Database Analytics course last semester. In this case, there are just 18 variables. Its a small feature set for today's machine learning libraries and computer capabilities. So we are ok. Specify in advance the number of principal components we want to use. Call the fit() method on the data frame.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(tb_deaths_per_100k)

In [ ]:
deaths_pca = pca.transform(tb_deaths_per_100k)

Create a dataframe using the two principal components generated.  

In [ ]:
# Create a dataframe with the two principal components generated. 
deaths_pca_df = pd.DataFrame(deaths_pca)

# Assign the indexes of tb_deaths_per_100 to deaths_pca_df. tb_deaths_per_100 has country names as its indexes. So they both
# will have same indexes now.
deaths_pca_df.index = tb_deaths_per_100k.index

# Rename the column names of deaths_pca_df to something meaningful. 
deaths_pca_df.columns = ['PC1','PC2']
deaths_pca_df.head()

You can look at explained variance ratio by each principal component as follows.

In [ ]:
print(pca.explained_variance_ratio_) 

The first component explains over 90% of the variance, while the second one accounts for nearly 6% for a total of almost 96% between the two of them.

Now that a lower dimensionality version of data is ready, call plot function on the data frame, by passing the kind of plot you want and what columns correspond to each axis. The for loop below adds an annotation to tag country with a point.

**Reference: ** [enumerate()](http://book.pythontips.com/en/latest/enumerate.html): It allows to loop over a list and have an automatic counter and value for corresponding to . Here is an example:

In [ ]:
%matplotlib inline

ax = deaths_pca_df.plot(kind='scatter', x='PC2', y='PC1', figsize=(16,8))

for i, country in enumerate(deaths_pca_df.index):
    ax.annotate(country, (deaths_pca_df.iloc[i].PC2, deaths_pca_df.iloc[i].PC1))

Create a bubble chart, by setting the point size to a value proportional to the mean value for all the years in that particular country. First of all, a new column containing the re-scaled mean per country across all the years shuld be added.

In [ ]:
# from sklearn.preprocessing import normalize

deaths_pca_df['country_mean'] = pd.Series(tb_deaths_per_100k.mean(axis=1), index=tb_deaths_per_100k.index)
country_mean_max = deaths_pca_df['country_mean'].max()
country_mean_min = deaths_pca_df['country_mean'].min()
country_mean_scaled = (deaths_pca_df.country_mean-country_mean_min) / country_mean_max

deaths_pca_df['country_mean_scaled'] = pd.Series(country_mean_scaled, index=deaths_pca_df.index) 
deaths_pca_df.head()

In [ ]:
deaths_pca_df.plot(kind='scatter', x='PC2', y='PC1', s=deaths_pca_df['country_mean_scaled']*100, figsize=(16,8), c=deaths_pca_df['country_mean_scaled'])

Above plots suggest that most variation happens along the y axis which is PC1. At the bottom of the chart there is concentration of countries that are mostly developed. When ascending the axis, the number of countries is becoming more sparse and they belong to less developed regions of the world.

When the points are coloured and sized using the magnitude average, the directions also correspond to a variation in these magnitudes.

PCA did tell us something about the data. How it is varying over all in the dataset. WE still dont know the relationships between countries. We will use k-means clustering to group countries based on how similar their situation has been year-by-year. Then we will use cluster assignment to colour code previous scatter plot that we generated.

When using k-means, the most important thing to do id to determine the right number of group for the data. This can be done more or less accurately by iterating through different values for the number of groups and compare an amount called the within-cluster sum of square distances for each iteration. This is the squared sum of distances to the cluster center for each cluster member. Of course this distance is minimal when the number of clusters gets equal to the number of samples, but we don't want to get there. We normally stop when the improvement in this value starts decreasing at a lower rate.

##### silhouette score

Silhouette refers to a method of interpretation and validation of consistency within clusters of data. The technique provides a succinct graphical representation of how well each object lies within its cluster - from wikipedia.org. The Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample. It will tell us whats is the optimum number of clsuters for given data. 

**Reference: **
- [sklearn.metrics.silhouette_score¶](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)
- [Silhouette (clustering)](https://en.wikipedia.org/wiki/Silhouette_(clustering))

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

s = []
for n in range(2,30):
    kmeans = KMeans(n_clusters=n)
    kmeans.fit(deaths_pca_df)

    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    s.append(silhouette_score(deaths_pca_df, labels, metric='euclidean'))

plt.plot(s)
plt.ylabel("Silouette")
plt.xlabel("k")
plt.title("Silouette for K-means cell's behaviour")
sns.despine()

The silhouette ranges from -1 to 1. A high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters. So we need to choose an optimum value. Lets start with 3 clusters.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit(tb_deaths_per_100k)

Store the cluster assignments together with each country in data frame. The cluster labels are returned in clusters.labels_.

In [ ]:
deaths_pca_df['cluster'] = pd.Series(clusters.labels_, index=deaths_pca_df.index)

And now we are ready to plot, using the cluster column as color.

In [ ]:
import numpy as np
from ggplot import *

# deaths_pca_df.plot(kind='scatter', x='PC2',y='PC1',c=deaths_pca_df.cluster.astype(np.float), figsize=(16,8))

# for i, country in enumerate(deaths_pca_df.index):
#     ax.annotate(cluster, (deaths_pca_df.iloc[i].PC2, deaths_pca_df.iloc[i].PC1))
    
ggplot(deaths_pca_df, aes(x='PC2', y='PC1', color='cluster')) + geom_point()

In [ ]:
tab = deaths_pca_df.groupby(['cluster']).size()
tab

In [ ]:
kmeans = KMeans(n_clusters=4)
clusters = kmeans.fit(tb_deaths_per_100k)

deaths_pca_df['cluster'] = pd.Series(clusters.labels_, index=deaths_pca_df.index)

ggplot(deaths_pca_df, aes(x='PC2', y='PC1', color='cluster')) + geom_point()

In [ ]:
tab = deaths_pca_df.groupby(['cluster']).size()
tab

In [ ]:
kmeans = KMeans(n_clusters=5)
clusters = kmeans.fit(tb_deaths_per_100k)
centroids = kmeans.cluster_centers_

deaths_pca_df['cluster'] = pd.Series(clusters.labels_, index=deaths_pca_df.index)

ggplot(deaths_pca_df, aes(x='PC2', y='PC1', color='cluster')) + geom_point()

In [ ]:
tab = deaths_pca_df.groupby(['cluster']).size()
tab

##### Interpreting cluster assignments

Cluster 1 contains 37 countries and the centroid of the cluster is shown below.

In [ ]:
centroids[0]

In [ ]:
print(deaths_pca_df.loc[deaths_pca_df['cluster']==0].shape)
deaths_pca_df[deaths_pca_df['cluster'] == 0].index.tolist()

Cluster 2 contains 143 countries and the centroid of the cluster is shown below.

In [ ]:
centroids[1]

In [ ]:
print(deaths_pca_df.loc[deaths_pca_df['cluster']==1].shape)
deaths_pca_df[deaths_pca_df['cluster'] == 1].index.tolist()

Cluster 3 contains just 1 country and the centroid of the cluster is shown below.

In [ ]:
centroids[2]

In [ ]:
print(deaths_pca_df.loc[deaths_pca_df['cluster']==2].shape)
deaths_pca_df[deaths_pca_df['cluster'] == 2].index.tolist()

Cluster 4 contains 23 countries and the centroid of the cluster is shown below.

In [ ]:
centroids[3]

In [ ]:
print(deaths_pca_df.loc[deaths_pca_df['cluster']==3].shape)
deaths_pca_df[deaths_pca_df['cluster'] == 3].index.tolist()

Cluster 5 contains 3 countries and the centroid of the cluster is shown below.

In [ ]:
centroids[4]

In [ ]:
print(deaths_pca_df.loc[deaths_pca_df['cluster']==4].shape)
deaths_pca_df[deaths_pca_df['cluster'] == 4].index.tolist()